In [9]:
import torch
import numpy as np
from src.preprocessing import remove_stop_words
from src.model_architectures.bert_classifier import SiameseNetwork

In [ ]:
net = torch.load('./models/0.2.0_BERT_epoch_3')
net.eval()

In [204]:
torch.save(net.state_dict(), './models/0.2.0_BERT_epoch_3_state.pt')

In [201]:
def inference():
    title1 = input('First title: ')
    title2 = input('Second title: ')
    
    title1 = remove_stop_words(title1)
    title2 = remove_stop_words(title2)
    
    data = np.array([title1, title2]).reshape(1, 2)
    forward = net(data)
    np_forward = forward.detach().numpy()[0]
    
    print('Output: {}'.format(torch.argmax(forward)))
    print('Softmax: Negative {:.4f}%, Positive {:.4f}%'.format(np_forward[0], np_forward[1]))

In [202]:
inference()

First title: lenovo yoga 11 3"" intel core i5 9600k 4 tb sata ssd 100 gb memory
Second title: lenovo yoga 11 3 inch laptop 4 tb sata ssd 66 gb ram core i5 9600k 3 7 ghz
Output: 0
Softmax: Negative 0.9024%, Positive 0.0976%


In [ ]:
lenovo yoga 11 3 inch laptop 4 tb sata ssd 66 gb ram core i5 9600k 3 7 ghz

In [ ]:
lenovo yoga 11 3"" intel core i5 9600k 4 tb sata ssd 100 gb memory

In [ ]:
16 gb ram 512 gb ssd asus zenbook ryzen cpu